In [1]:
import json

In [10]:
path1 = "./dynamic/test_output/captions/test/"
path2 = "./transformer/transformer_self_att_8_CLEVR_CHANGE_nsc_results.json"
path_gt = "../data/total_change_captions_reformat.json"

In [11]:
with open(path2,"r") as f:
    tfimages = json.load(f)

In [12]:
tfimages

[{'image_id': '006594.png_n', 'caption': 'there is no change'},
 {'image_id': '006580.png_n', 'caption': 'there is no change'},
 {'image_id': '002360.png_n', 'caption': 'there is no change'},
 {'image_id': '002714.png_n', 'caption': 'there is no change'},
 {'image_id': '000992.png_n', 'caption': 'there is no change'},
 {'image_id': '007766.png_n', 'caption': 'there is no change'},
 {'image_id': '005483.png_n', 'caption': 'there is no change'},
 {'image_id': '000979.png_n', 'caption': 'there is no change'},
 {'image_id': '007114.png_n', 'caption': 'there is no change'},
 {'image_id': '004556.png_n', 'caption': 'there is no change'},
 {'image_id': '006522.png_n', 'caption': 'there is no change'}]

In [3]:
with open(path1+"nsc_results.json","r") as f:
    listimages = json.load(f)

In [4]:
len(listimages)

3985

In [5]:
with open(path1+"sc_results.json","r") as f:
    scimages = json.load(f)

In [6]:
len(scimages)

3985

In [7]:
sc_ids = [im['image_id'] for im in scimages]

In [8]:
nsc_ids = [im['image_id'] for im in listimages]

In [9]:
set(nsc_ids) & set(sc_ids)

set()

In [13]:
with open(path_gt,"r") as f:
    gtdata = json.load(f)

In [14]:
gtdata["annotations"]

[{'caption': 'the tiny matte thing turned metal',
  'id': 0,
  'image_id': '011846.png'},
 {'caption': 'the red rubber object became shiny',
  'id': 1,
  'image_id': '011846.png'},
 {'caption': 'the small block became shiny',
  'id': 2,
  'image_id': '011846.png'},
 {'caption': 'the small block turned shiny',
  'id': 3,
  'image_id': '011846.png'},
 {'caption': 'the small red matte block that is in front of the large blue block changed to metallic',
  'id': 4,
  'image_id': '011846.png'},
 {'caption': 'the small red matte block that is in front of the big blue matte thing became metal',
  'id': 5,
  'image_id': '011846.png'},
 {'caption': 'the red thing became metallic',
  'id': 6,
  'image_id': '011846.png'},
 {'caption': 'the small red matte block in front of the blue matte thing turned metallic',
  'id': 7,
  'image_id': '011846.png'},
 {'caption': 'the large cylinder is missing',
  'id': 8,
  'image_id': '031910.png'},
 {'caption': 'the green cylinder is no longer there',
  'id': 9

In [79]:
class ClevrGrammar:
    
    def __init__(self):
        self.shapes = set(['ball', 'block', 'cube', 'cylinder', 'sphere'])
        self.colors = set(['red', 'cyan', 'brown', 'blue', 'purple', 'green', 'gray', 'yellow'])
        self.materials = set(['metallic', 'matte', 'rubber', 'shiny', 'metal'])
        self.verb_types = [("nochange",["remains the same","no change","same as before"]),\
                           ("add",["has been added","has appeared","has been newly placed"]),\
                           ("drop",["is no longer there","is missing","is gone","has disappeared"]),\
                           ("loc",["is in a different location","changed its location","moved"]),\
                           ("color:material",["became","turned","changed to"])]
       
        
   
    def parse_caption(self,caption):
        
        parse = {"type":None,
        "change_att":None,
        "target_shape":None,
        "target_color":None,
        "target_material":None, 
        "lm_shape":None,
        "lm_color":None,
        "lm_material":None}
        
        tokens = caption.split()
        subject_caption = None
        landmark_caption = None
        
        for (vtype,vlist) in self.verb_types:
            for v in vlist:
                if v in caption:
                    parse['type'] = vtype
                    subject_caption = caption[:caption.index(v)]
                    #print(caption.index(v))
                    break
                    
        if parse['type'] == "color:material":
            if tokens[-1] in self.colors:
                parse['change_att'] = tokens[-1]
                parse['type'] = "color"
            elif tokens[-1] in self.materials:
                parse['change_att'] = tokens[-1]
                parse['type'] = "material"
            else:
                print("WARNING, unexpected last token",caption)
                
        if parse['type'] == "nochange":
            return parse
                
        if not parse["type"]:
            print("WARNING, no change type",caption)
                
        if "that is" in subject_caption:
            landmark_caption = subject_caption[subject_caption.index("that is"):]
            subject_caption = subject_caption[:subject_caption.index("that is")]
        
            
        for subj_tok in subject_caption.split():
            if subj_tok in self.colors:
                parse["target_color"] = subj_tok
            elif subj_tok in self.shapes:
                parse["target_shape"] = subj_tok
            elif subj_tok in self.materials:
                parse["target_material"] = subj_tok
                
        if landmark_caption:
            for subj_tok in landmark_caption.split():
                if subj_tok in self.colors:
                    parse["lm_color"] = subj_tok
                elif subj_tok in self.shapes:
                    parse["lm_shape"] = subj_tok
                elif subj_tok in self.materials:
                    parse["lm_material"] = subj_tok
                
            
        return parse
                    
            
        
        


In [80]:
c = "the small blue matte ball that is on the left side of the cyan sphere has been added"

In [81]:
g = ClevrGrammar()

In [82]:
g.parse_caption(c)

{'type': 'add',
 'change_att': None,
 'target_shape': 'ball',
 'target_color': 'blue',
 'target_material': 'matte',
 'lm_shape': 'sphere',
 'lm_color': 'cyan',
 'lm_material': None}

In [86]:
for sc in scimages:
    thisp = g.parse_caption(sc['caption'])
    print(sc['image_id'],thisp['type'])

019316.png add
030122.png drop
001697.png color
001882.png color
003758.png color
012919.png material
009369.png material
039433.png color
024681.png drop
001554.png color
006093.png nochange
022977.png add
027020.png drop
032926.png loc
033451.png loc
009856.png material
038276.png nochange
016567.png add
004292.png color
009896.png nochange
025565.png material
018493.png add
019667.png add
039817.png loc
012463.png material
008410.png material
010468.png material
037063.png loc
035021.png loc
028774.png drop
021395.png add
021670.png add
024635.png drop
037240.png loc
004586.png color
039732.png nochange
004006.png color
026548.png drop
024783.png drop
035156.png nochange
001621.png color
015982.png material
003588.png color
035818.png nochange
024380.png drop
011700.png material
010723.png nochange
028997.png drop
019540.png add
036869.png nochange
038196.png loc
023369.png add
002804.png color
032767.png nochange
005546.png color
032004.png drop
003585.png color
026446.png drop
022

025074.png drop
017195.png add
022971.png add
027740.png drop
019557.png add
007397.png color
034639.png loc
018556.png add
016385.png add
029993.png nochange
014855.png material
035420.png loc
002191.png color
008766.png material
025427.png drop
039428.png loc
020192.png add
018112.png add
014590.png material
014258.png material
039193.png nochange
002140.png color
030318.png drop
037529.png loc
023607.png add
011913.png nochange
030320.png drop
011269.png material
015446.png nochange
025828.png drop
019860.png add
029299.png drop
002083.png color
004317.png color
013296.png material
022721.png add
016069.png add
003577.png color
032655.png nochange
004138.png color
008369.png material
031955.png nochange
028003.png drop
014584.png material
018129.png add
021596.png add
006636.png color
035312.png nochange
002456.png color
019783.png nochange
032308.png drop
016241.png nochange
032896.png loc
035522.png loc
032156.png drop
009130.png nochange
000868.png color
001109.png color
018110.p

013441.png nochange
018653.png add
039992.png loc
006302.png color
011161.png material
021156.png add
003531.png color
007022.png nochange
004118.png color
039673.png loc
035793.png nochange
028322.png drop
015833.png nochange
015784.png material
025034.png drop
010480.png material
012381.png material
011557.png nochange
009231.png material
000715.png color
014819.png material
007288.png color
006209.png color
026464.png drop
009356.png material
021744.png add
009587.png material
032438.png loc
012546.png material
036841.png add
033042.png loc
017436.png add
009978.png material
008831.png material
023397.png add
016508.png add
008170.png material
007911.png color
016080.png add
020930.png add
034910.png nochange
038129.png nochange
039065.png nochange
013060.png material
029633.png drop
003561.png color
004718.png color
006389.png nochange
029048.png nochange
013379.png material
031129.png drop
003701.png color
028706.png drop
022249.png add
005271.png color
027241.png drop
006395.png 

021172.png add
006040.png color
030194.png drop
018247.png add
029598.png drop
000023.png color
036134.png drop
002502.png color
009948.png nochange
017087.png add
039926.png nochange
038097.png loc
013098.png material
022171.png add
004620.png color
007328.png color
037234.png nochange
038588.png loc
011192.png material
032723.png loc
024240.png drop
026713.png drop
011281.png material
016910.png add
010771.png material
009535.png material
003084.png color
006125.png color
006747.png color
023106.png add
013264.png nochange
020895.png nochange
023461.png add
011303.png material
032525.png loc
017347.png add
029470.png drop
000425.png color
025541.png loc
012724.png material
027699.png drop
026128.png nochange
004166.png color
034998.png nochange
015267.png material
026578.png drop
037255.png loc
010824.png material
031311.png drop
037610.png drop
002399.png color
006977.png color
036156.png nochange
030118.png drop
013255.png material
028117.png drop
000489.png color
029431.png drop
0

In [92]:
gt_parses = {}
for gt_item in gtdata['annotations']:
    if gt_item['image_id'] in sc_ids:
        if gt_item['image_id'] not in gt_parses:
            gt_parses[gt_item['image_id']] = []
        gt_parses[gt_item['image_id']].append((gt_item['caption'],g.parse_caption(gt_item['caption'])))

In [113]:
def get_att_acc(pred,gold,att):
    
    if pred[att]:
        for _,pg in gold:
            if pred[att] == pg[att]:
                return 1
    return 0

def has_att(pred,att):
    
    if pred[att]:
        return 1
    return 0

In [115]:

accs = {"type":0,
        "change_att":0,
        "target_shape":0,
        "target_color":0,
        "target_material":0, 
        "lm_shape":0,
        "lm_color":0,
        "lm_material":0}
totals = {"type":0,
        "change_att":0,
        "target_shape":0,
        "target_color":0,
        "target_material":0, 
        "lm_shape":0,
        "lm_color":0,
        "lm_material":0}


for sc in scimages:
    thisp = g.parse_caption(sc['caption'])
    thisim = sc['image_id']
    gttype = gt_parses[thisim][0][1]['type']
    for key_acc in accs:
        accs[key_acc] += get_att_acc(thisp, gt_parses[thisim],key_acc)
        totals[key_acc] += has_att(thisp,key_acc)
        
for key_acc in accs:
    print(key_acc,accs[key_acc]/totals[key_acc])
#print(coloracc,coloracc/len(scimages))

type 0.7884567126725219
change_att 0.9373156342182891
target_shape 0.9450516089860352
target_color 0.9882139619220308
target_material 0.8811188811188811
lm_shape 0.37577639751552794
lm_color 0.24059920513604402
lm_material 0.23500165727543917
